In [1]:
import pandas as pd
import numpy as np
import glob
#import nibabel as nib
from scipy.ndimage import label
from numpy.linalg import inv

## msid + mseid data

In [32]:
#Construct data table
final_cols = ["msid",
             "tot_years_gt1_lesion_present",
             "inf_years_gt1_lesion_present",
             "jux_years_gt1_lesion_present",
             "per_years_gt1_lesion_present",
             "sub_years_gt1_lesion_present",
             "tot_lesion_presentation",
             "inf_lesion_presentation",
             "jux_lesion_presentation",
             "per_lesion_presentation",
             "sub_lesion_presentation",
             "comparison",
             "tot_lesion_change_percent",
             "tot_lesion_change_absolute",
             "tot_lesion_vol_change_percent",
             "tot_lesion_vol_change_absolute",
             "inf_lesion_change_percent",
             "inf_lesion_change_absolute",
             "inf_lesion_vol_change_percent",
             "inf_lesion_vol_change_absolute",
             "jux_lesion_change_percent",
             "jux_lesion_change_absolute",
             "jux_lesion_vol_change_percent",
             "jux_lesion_vol_change_absolute",
             "per_lesion_change_percent",
             "per_lesion_change_absolute",
             "per_lesion_vol_change_percent",
             "per_lesion_vol_change_absolute",
             "sub_lesion_change_percent",
             "sub_lesion_change_absolute",
             "sub_lesion_vol_change_percent",
             "sub_lesion_vol_change_absolute",
             "tot_number_increasing_0",
             "tot_number_increasing_10",
             "tot_number_increasing_20",
             "tot_number_increasing_50",
             "tot_number_increasing_100",
             "inf_number_increasing_0",
             "inf_number_increasing_10",
             "inf_number_increasing_20",
             "inf_number_increasing_50",
             "inf_number_increasing_100",
             "jux_number_increasing_0",
             "jux_number_increasing_10",
             "jux_number_increasing_20",
             "jux_number_increasing_50",
             "jux_number_increasing_100",
             "per_number_increasing_0",
             "per_number_increasing_10",
             "per_number_increasing_20",
             "per_number_increasing_50",
             "per_number_increasing_100",
             "sub_number_increasing_0",
             "sub_number_increasing_10",
             "sub_number_increasing_20",
             "sub_number_increasing_50",
             "sub_number_increasing_100",
             "tot_number_decreasing_0",
             "tot_number_decreasing_10",
             "tot_number_decreasing_20",
             "tot_number_decreasing_50",
             "tot_number_decreasing_100",
             "inf_number_decreasing_0",
             "inf_number_decreasing_10",
             "inf_number_decreasing_20",
             "inf_number_decreasing_50",
             "inf_number_decreasing_100",
             "jux_number_decreasing_0",
             "jux_number_decreasing_10",
             "jux_number_decreasing_20",
             "jux_number_decreasing_50",
             "jux_number_decreasing_100",
             "per_number_decreasing_0",
             "per_number_decreasing_10",
             "per_number_decreasing_20",
             "per_number_decreasing_50",
             "per_number_decreasing_100",
             "sub_number_decreasing_0",
             "sub_number_decreasing_10",
             "sub_number_decreasing_20",
             "sub_number_decreasing_50",
             "sub_number_decreasing_100",
             "tot_new_lesions",
             "tot_lost_lesions",
             "inf_new_lesions",
             "inf_lost_lesions",
             "jux_new_lesions",
             "jux_lost_lesions",
             "per_new_lesions",
             "per_lost_lesions",
             "sub_new_lesions",
             "sub_lost_lesions",
             "lesion",
             "in_both",
             "overlap",
             "volume_change",
             "change_in_lesion_type"]
analysis = {}
for item in final_cols:
    analysis[item] = []
prev_data = pd.DataFrame.from_csv('clinical_info.csv')

In [54]:
#created matrix of tps
tp_matrix = {}
count = 1
for x in range(1,10):
    for y in range(x+1,11):
        tp_matrix["%s,%s" %(x,y)]=count
        count += 1
tp_matrix

#create sorted DataFrame in order of msid and tp
#create list of msids for looping purposes
df = pd.DataFrame(prev_data).sort_values(by=['msid','tp']).reset_index()
cols = pd.Index(df.columns)
msids = df.msid.values
types = ['tot','inf','jux','per','sub']

#percent change function
def percentchange(num2,num1):
    if num1 == 0 or num2 == 0:
        value = 0
    else:
        value = ((float(num2)-float(num1))/(float(num1))) * 100
    return value

#meat of the program
for ids in msids:
    #collect values per tp per msid 
    #specifically collect mseids, total lesion count, total lesion volume, total lesion count (type), total lesion volume (type)
    msid_vals = {}
    current_msid = df['msid']==ids
    tp_list = df[current_msid]['tp'].values 
    mseid_list = df[current_msid]['mse'].values
    for les in types:
        msid_vals['%s_tot_list' % les] = df[current_msid]['%s_total_lesions' % les].values
        msid_vals['%s_vol_list' % les] = df[current_msid]['%s_total_volume_of_lesions' % les].values    
    
    #calculate number of year by year comparisons needed for each msid
    tp_yr_comp = []
    tp_ix_comp = []
    for yrx in range(len(tp_list)):
        for yry in range(y+1,len(tp_list)):
            tp_yr_comp.append("%s,%s" % (tp_list[yrx],tp_list[yry]))
            tp_ix_comp.append([yrx,yry])
    
    #print values for debugging purposes
    print ids,tp_list[0],mseid_list[0],msid_vals['tot_tot_list']

ms0056 1 mse1328 [11]
ms0060 1 mse1334 [2]
ms0064 1 mse1337 [14]
ms0069 1 mse1343 [14]
ms0070 1 mse1349 [37]
ms0071 1 mse1355 [19]
ms0073 1 mse1361 [4]
ms0077 1 mse1367 [19]
ms0078 1 mse1373 [25]
ms0086 1 mse1379 [15]
ms0109 1 mse1384 [38]
ms0135 1 mse1385 [4]
ms0165 1 mse1391 [11]
ms0170 1 mse1 [15]
ms0171 1 mse1399 [6]
ms0172 1 mse1406 [17]
ms0173 1 mse1411 [9]
ms0174 1 mse1416 [71]
ms0175 1 mse1421 [4]
ms0176 1 mse3576 [20]
ms0179 1 mse1430 [15]
ms0181 1 mse1436 [3]
ms0184 1 mse1442 [4]
ms0185 1 mse1448 [3]
ms0195 1 mse1449 [12]
ms0198 1 mse1455 [12]
ms0227 1 mse1460 [36]
ms0230 1 mse1463 [9]
ms0241 1 mse1468 [23]
ms0242 1 mse1474 [17]
ms0243 1 mse1480 [7]
ms0245 1 mse1485 [15]
ms0247 1 mse1491 [26]
ms0248 1 mse1497 [34]
ms0249 1 mse1500 [81]
ms0270 1 mse1505 [18]
ms0282 1 mse1511 [38]
ms0286 1 mse1517 [45]
ms0376 1 mse1520 [41]
ms0378 1 mse1526 [37]
ms0386 1 mse1532 [18]
ms0387 1 mse1537 [11]
ms0388 1 mse1543 [8]
ms0405 1 mse1547 [5]
ms0407 1 mse1553 [34]
ms0408 1 mse1554 [7]
ms042

In [204]:
#created test data and dataframe, sorted by mkid and tp
test = {"mkid" : ["mk1","mk2","mk3","mk4","mk1","mk2","mk3","mk4","mk1","mk2","mk3","mk4"],
        "mkeid" : ["mke009","mke002","mke003","mke004","mke005","mke006","mke007","mke008",
                   "mke001","mke010","mke011","mke012"],
        "tp" : [1,1,1,1,2,2,2,3,3,3,5,4],
        "tot_total_lesions" : [5,10,6,14,1,11,24,13,25,27,10,4],
        "sub_total_lesions" : [1,5,0,5,1,0,11,3,12,9,3,2],
        "inf_total_lesions" : [1,2,2,3,0,1,4,3,25,4,3,0],
        "per_total_lesions" : [2,2,4,4,0,5,10,3,4,10,3,1],
        "jux_total_lesions" : [1,1,0,4,0,4,0,4,5,4,1,1],
       }
testcolumns = test.keys()
testdf = pd.DataFrame(test, columns=[testcolumns]).sort_values(by=['mkid','tp'])
testids = testdf.mkid.values
types = ['tot','inf','jux','per','sub']


#analysis dataframe
analysiscol=["mkid",
             "comparison",
             "tot_lesion_change_percent",
             "tot_lesion_change_absolute",
             "inf_lesion_change_percent",
             "inf_lesion_change_absolute",
             "jux_lesion_change_percent",
             "jux_lesion_change_absolute",
             "per_lesion_change_percent",
             "per_lesion_change_absolute",
             "sub_lesion_change_percent",
             "sub_lesion_change_absolute"]
analysis = {}
for item in analysiscol:
    analysis[item] = []

#perform calculations
print "mkid", "comparison", "abs_lesion_change", "per_lesion_change"
for x in range(0,len(testids),3):
    msid_vals = {}
    val = testdf['mkid']==testids[x] #shortcut to gathering variables by mkid
    tp_list = testdf[val]['tp'].values 
    mkeid_list = testdf[val]['mkeid'].values
    tp_yr_comp = []
    tp_ix_comp = []
    for yrx in range(len(tp_list)):
        for yry in range(yrx+1,len(tp_list)):
            tp_yr_comp.append("%s,%s" % (tp_list[yrx],tp_list[yry]))
            tp_ix_comp.append([yrx,yry])
    for les in range(len(types)):
        msid_vals['%s_tot_list' % types[les]] = testdf[val]['%s_total_lesions' % types[les]].values
        msid_vals['%s_change_list' % types[les]] = []
        for comp in range(len(tp_ix_comp)):
            les_change_abs = msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][1]] - msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][0]]
            les_change_per = percentchange(msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][1]],msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][0]])
            msid_vals['%s_change_list' % types[les]].append([les_change_abs,les_change_per])
    #print testids[x] 
    #for les in types:
    #    for y in range(len(msid_vals['%s_change_list' % les])):
    #        print les,tp_ix_comp[y], msid_vals['%s_change_list' % les][y]
    #print
    print len(analysiscol),len(types),len(msid_vals['tot_change_list'])
    for count in range(len(msid_vals['tot_change_list'])):
        analysis["tot_lesion_change_percent"].append(msid_vals['tot_change_list'][count][1])
        analysis["tot_lesion_change_absolute"].append(msid_vals['tot_change_list'][count][0])
        analysis["sub_lesion_change_percent"].append(msid_vals['sub_change_list'][count][1])
        analysis["sub_lesion_change_absolute"].append(msid_vals['sub_change_list'][count][0])
        analysis["per_lesion_change_percent"].append(msid_vals['per_change_list'][count][1])
        analysis["per_lesion_change_absolute"].append(msid_vals['per_change_list'][count][0])
        analysis["jux_lesion_change_percent"].append(msid_vals['jux_change_list'][count][1])
        analysis["jux_lesion_change_absolute"].append(msid_vals['jux_change_list'][count][0])
        analysis["inf_lesion_change_percent"].append(msid_vals['jux_change_list'][count][1])
        analysis["inf_lesion_change_absolute"].append(msid_vals['jux_change_list'][count][0])
        analysis["mkid"].append(testids[x])
        analysis["comparison"].append(tp_yr_comp[count])
lulz = pd.DataFrame(analysis,columns=analysiscol)
lulz

mkid comparison abs_lesion_change per_lesion_change
12 5 3
12 5 3
12 5 3
12 5 3


,mkid,comparison,tot_lesion_change_percent,tot_lesion_change_absolute,inf_lesion_change_percent,inf_lesion_change_absolute,jux_lesion_change_percent,jux_lesion_change_absolute,per_lesion_change_percent,per_lesion_change_absolute,sub_lesion_change_percent,sub_lesion_change_absolute
0,mk1,"1,2",-80.000000,-4,0.0,-1,0.0,-1,0.000000,-2,0.000000,0
1,mk1,"1,3",400.000000,20,400.0,4,400.0,4,100.000000,2,1100.000000,11
2,mk1,"2,3",2400.000000,24,0.0,5,0.0,5,0.000000,4,1100.000000,11
3,mk2,"1,2",10.000000,1,300.0,3,300.0,3,150.000000,3,0.000000,-5
4,mk2,"1,3",170.000000,17,300.0,3,300.0,3,400.000000,8,80.000000,4
5,mk2,"2,3",145.454545,16,0.0,0,0.0,0,100.000000,5,0.000000,9
6,mk3,"1,2",300.000000,18,0.0,0,0.0,0,150.000000,6,0.000000,11
7,mk3,"1,5",66.666667,4,0.0,1,0.0,1,-25.000000,-1,0.000000,3
8,mk3,"2,5",-58.333333,-14,0.0,1,0.0,1,-70.000000,-7,-72.727273,-8
9,mk4,"1,3",-7.142857,-1,0.0,0,0.0,0,-25.000000,-1,-40.000000,-2
